In [1]:
import os
import import_ipynb
import rfa_utils.general_fns as gf
import rfa_utils.clean_crypto as cc
import rfa_utils.coingecko_api as ca
import rfa_utils.crypto_scrape as cs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

importing Jupyter notebook from c:\Users\nguye\Documents\Sonderium\Projects\Remittance-Fees-Analysis\rfa_utils\general_fns.ipynb
importing Jupyter notebook from c:\Users\nguye\Documents\Sonderium\Projects\Remittance-Fees-Analysis\rfa_utils\clean_crypto.ipynb
importing Jupyter notebook from c:\Users\nguye\Documents\Sonderium\Projects\Remittance-Fees-Analysis\rfa_utils\coingecko_api.ipynb
importing Jupyter notebook from c:\Users\nguye\Documents\Sonderium\Projects\Remittance-Fees-Analysis\rfa_utils\crypto_scrape.ipynb


In [2]:
# List of crypto symbols and IDs
syms = ['btc', 'xrp', 'doge', 'ltc', 'xmr', 'bch', 'xlm', 'bsv', 'zec', 'dash']
cids = ['bitcoin', 'ripple', 'dogecoin', 'litecoin', 'monero', 'bitcoin-cash', 'stellar', 'bitcoin-cash-sv', 'zcash', 'dash']

# Variables for API call
currency = 'usd'
start_date = '2019/01/01'
end_date = '2022/12/31'

In [3]:
# Check and create API data
api_path = os.path.join('data/crypto/', 'api-data.xlsx')
gf.check_and_create(api_path, ca.create_api_dict, syms, cids, currency, start_date, end_date)

In [4]:
# Set up Chrome headless options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

# Set up the webdriver with the Chrome options
driver = webdriver.Chrome(options=chrome_options)

In [5]:
# Dict of token stats list for scraping
token_stats = {
    'btc': ['bitcoin-transactions', 'bitcoin-transactionvalue', 'bitcoin-mediantransactionvalue', 'bitcoin-transactionfees', 'bitcoin-median_transaction_fee'],
    'xrp': ['xrp-transactions', 'transactionfees-xrp'],
    'doge': ['dogecoin-transactions', 'dogecoin-transactionvalue', 'dogecoin-mediantransactionvalue', 'dogecoin-transactionfees', 'dogecoin-median_transaction_fee'],
    'ltc': ['litecoin-transactions', 'litecoin-transactionvalue', 'litecoin-mediantransactionvalue', 'litecoin-transactionfees', 'litecoin-median_transaction_fee'],
    'xmr': ['monero-transactions', 'monero-transactionfees'],
    'bsv': ['transactions-bsv'],
    'bch': ['bitcoin%20cash-transactions', 'bitcoin%20cash-transactionvalue', 'bitcoin%20cash-mediantransactionvalue', 'bitcoin%20cash-transactionfees', 'bitcoin%20cash-median_transaction_fee'],
    'zec': ['zcash-transactions', 'zcash-transactionfees', 'transactionvalue-zec', 'mediantransactionvalue-zec'],
    'dash': ['dash-transactions', 'dash-transactionfees', 'dash-median_transaction_fee', 'transactionvalue-dash', 'mediantransactionvalue-dash']
}

In [6]:
# Check and create scraped data
scrape_path = os.path.join('data/crypto/', 'scraped-data.xlsx')
gf.check_and_create(scrape_path, cs.create_scrape_dict, token_stats, driver, start_date, end_date)

In [7]:
# Check and create combined Excel file
combined_path = os.path.join('data/crypto/', 'combined-data.xlsx')
if not os.path.exists(combined_path):
    # List of scraped Excel sheets
    scraped_sheets = ['btc', 'xrp_bit', 'xrp_mes', 'doge', 'ltc', 'xmr', 'bsv_bit', 'bsv_mes', 'bch', 'xlm', 'zec', 'dash']
    # Read filled Excel sheets into dataframes
    filled_api = gf.import_excel(
        'data/crypto/filled-api-data.xlsx',
        syms
    )
    filled_scraped = gf.import_excel(
        'data/crypto/filled-scraped-data.xlsx',
        scraped_sheets
    )
    gf.check_and_create(combined_path, cc.merge_api_scraped, filled_api, filled_scraped, syms)

In [9]:
# Check and create imputed file
imputed_path = os.path.join('data/crypto/', 'imputed-data.xlsx')
if not os.path.exists(imputed_path):
    combined = gf.import_excel(
        combined_path,
        syms
    )
    # Impute missing columns with miceforest
    gf.check_and_create(imputed_path, cc.impute_dfs, combined)